In [ ]:
# Data Preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
training_dataset = pd.read_csv('datasets/Google_Stock_Price_Train.csv')
training_dataset.head()

In [ ]:
X = training_dataset.iloc[:, 1:2].values
X

In [ ]:
# min-max scalar to normalize the values in X from 0 to 1
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0,1))
scaled_X = mms.fit_transform(X)
scaled_X

In [ ]:
X_train = []
y_train = []

for i in range(60, 1258):
    X_train.append(scaled_X[i-60:i, 0])
    y_train.append(scaled_X[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import LSTM
from keras.layers import Dropout

regressor = Sequential()
# 1st layer
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2)) 
# 2nd layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2)) 
# 3rd layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2)) 
# 4th layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2)) 
# Last layer output
regressor.add(Dense(units=1))


In [ ]:
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
regressor.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
testing_dataset= pd.read_csv('datasets\Google_Stock_Price_Test.csv')
real_stock_price = testing_dataset.iloc[:, 1:2].values

In [ ]:
dataset_total = pd.concat((training_dataset['Open'], testing_dataset['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(testing_dataset) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = mms.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = mms.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()